# Query and analyze NEON discharge and water chemistry data from BigQuery

Funded by the National Science Foundation (NSF) and proudly operated by Battelle, the [National Ecological Observatory Network (NEON)](https://www.neonscience.org/) program provides open, continental-scale data across the United States that characterize and quantify complex, rapidly changing ecological processes. The Observatory’s comprehensive design supports greater understanding of ecological change and enables forecasting of future ecological conditions. NEON collects and processes data from field sites located across the continental U.S., Puerto Rico, and Hawaii over a 30-year timeframe. NEON provides free and open data that characterize plants, animals, soil, nutrients, freshwater, and the atmosphere. These data may be combined with external datasets or data collected by individual researchers to support the study of continental-scale ecological change.

This notebook is a demonstration of accessing and querying NEON data stored in Google Cloud BigQuery. Only a small subset of NEON data are currently available in this format, but more may be added in the future. The example datasets used are [Continuous discharge (DP4.00130.001)](https://data.neonscience.org/data-products/DP4.00130.001/RELEASE-2022) and [Chemical properties of surface water (DP1.20093.001)](https://data.neonscience.org/data-products/DP1.20093.001/RELEASE-2022). The RELEASE-2022 versions of both data products are available in BigQuery tables via the Analytics Hub. In both cases, we'll be focusing on the critical data table (csd_continuousDischarge and swc_externalLabDataByAnalyte, respectively) but other data and metadata tables are also available in the Analytics Hub listing. For more details about the two data products, see the product details pages at the links above, and the variables tables in each Analytics Hub listing.

These two data products can be used together to estimate nutrient fluxes in stream ecosystems. In this notebook we first demonstrate a query on each data product independently, then a query that brings the two together to estimate dissolved organic carbon fluxes across NEON's stream sites.

### Setup

Install the Plotly Express module for visualization, and import other needed libraries.

In [ ]:
pip install plotly_express

In [ ]:
import matplotlib.pyplot as plt
import pandas
import requests
import json
import plotly.express as px
import os

Set environment variable to the project you're working in.

In [ ]:
os.environ['GCP_PROJECT'] = 'neon-sandbox-clunch-a4bc'

### Query continuous discharge data

Query for Continuous discharge (DP4.00130.001) data from the RELEASE-2022 dataset in BigQuery. `%%bigquery` is the "magic" command that designates the entire cell as one query command. `csd_core` is the name of the object to write the output to, and the query itself is written in SQL.

Here, we'll download the site, date, discharge rate, and the two-standard-deviation range for all data in RELEASE-2022, filtering out any records with the final quality flag raised, or where a discharge value is not populated.

In [ ]:
%%bigquery csd_core
SELECT
    siteID,
    endDate,
    maxpostDischarge,
    withRemnUncQLower2Std,
    withRemnUncQUpper2Std
FROM
    `neon_continuous_discharge.csd_continuousDischarge`
WHERE
    dischargeFinalQF = 0 AND 
    maxpostDischarge IS NOT NULL
ORDER BY
    siteID, endDate;

Take a look at the first 10 rows to make sure the data look as expected:

In [ ]:
csd_core[0:9]

The numeric data are imported from BigQuery as type "decimal", which `pandas` doesn't work with correctly. Convert the format.

In [ ]:
csd_core.maxpostDischarge = csd_core.maxpostDischarge.astype("float")
csd_core.withRemnUncQLower2Std = csd_core.withRemnUncQLower2Std.astype("float")
csd_core.withRemnUncQUpper2Std = csd_core.withRemnUncQUpper2Std.astype("float")

Start by subsetting to a single site, to explore the data before proceeding to larger analyses. Let's look at Como Creek (COMO), in Colorado.

In [ ]:
csd_COMO = csd_core[csd_core["siteID"] == "COMO"]

Start by plotting the data for all time at COMO, including the upper and lower uncertainty bounds.

In [ ]:
plt.plot(csd_COMO.endDate, 
         csd_COMO.maxpostDischarge,
         linewidth=0.25, color="black")
plt.fill_between(csd_COMO.endDate, 
                 csd_COMO.withRemnUncQLower2Std, 
                 csd_COMO.withRemnUncQUpper2Std,
                 color="gray", alpha=0.2)

There are three years of data for COMO in RELEASE-2022. This stream is in the Rocky Mountains, and we can see clearly that there is a long period each winter when flow is extremely low or zero, with almost all the flow happening in a few months. NEON covers a very broad range of ecosystem types, so we expect to see very different patterns at different sites.

### Query Chemical properties of surface water data

Now let's get the water chemistry data. For this demo, we'll be focusing on dissolved organic carbon (DOC), so we can select just that analyte from the RELEASE-2022 dataset in BigQuery.

In [ ]:
%%bigquery swc_doc
SELECT
    siteID,
    collectDate,
    analyteConcentration AS docConcentration
FROM
    `neon_chemical_properties_of_surface_water.swc_externalLabDataByAnalyte`
WHERE
    analyte = "DOC"
ORDER BY
    siteID, collectDate;

Again, check the first few rows:

In [ ]:
swc_doc[0:9]

In [ ]:
swc_doc.docConcentration = swc_doc.docConcentration.astype("float")

And plot DOC concentration for COMO on the right-hand axis along with the discharge data:

In [ ]:
doc_COMO = swc_doc[swc_doc["siteID"] == "COMO"]

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(csd_COMO.endDate, 
         csd_COMO.maxpostDischarge,
         linewidth=0.25, color="black")
ax1.fill_between(csd_COMO.endDate, 
                 csd_COMO.withRemnUncQLower2Std, 
                 csd_COMO.withRemnUncQUpper2Std,
                 color="gray", alpha=0.2)
ax2.scatter(doc_COMO.collectDate,
            doc_COMO.docConcentration,
            marker="o", color="blue", s=0.5)

We learn a couple of things from this figure: first, DOC concentrations rise and fall in parallel with discharge rate,  and second, there are chemistry samples collected from time periods when discharge estimates aren't available. Of course, since discharge estimates are made every minute, there are also many discharge estimates that don't have corresponding DOC samples.

In order to calculate the flux of DOC, we need to do some alignment of the two datasets. The end result we want is a series of sites and dates, each with a corresponding discharge value and DOC value. To get there, we need to:
* Subset the chemistry data to only the sites contained in the discharge data (no discharge at lake sites)
* Match the dates between the chemistry and discharge data
* Calculate a discharge value for the time period represented by each DOC collection

### Query both data products together and join

We could do this by transforming the `pandas` data frames we already have, and that would be a reasonable approach. But we're here to explore the capabilities of BigQuery. It enables us to calculate the average discharge on each day of each year and join the data to the DOC data from the days it was collected, using SQL. Using this approach, we only access the subsetted, aggregated data, rather than downloading the entirety of both datasets.

In [ ]:
%%bigquery flux
SELECT
*
FROM
    (SELECT
        siteID,
        EXTRACT(YEAR FROM endDate) AS yr,
        EXTRACT(DAYOFYEAR FROM endDate) AS doy,
        AVG(maxpostDischarge) AS dischargeMean
    FROM
        `neon_continuous_discharge.csd_continuousDischarge`
    WHERE
        dischargeFinalQF = 0 AND 
        maxpostDischarge IS NOT NULL
    GROUP BY
        siteID, 
        EXTRACT(YEAR FROM endDate),
        EXTRACT(DAYOFYEAR FROM endDate)) csd
JOIN
    (SELECT
        siteID,
        EXTRACT(YEAR FROM collectDate) AS yr,
        EXTRACT(DAYOFYEAR FROM collectDate) AS doy,
        analyteConcentration AS doc
    FROM
        `neon_chemical_properties_of_surface_water.swc_externalLabDataByAnalyte`
    WHERE
        analyte = "DOC") swc
ON
    csd.siteID = swc.siteID AND
    csd.yr = swc.yr AND
    csd.doy = swc.doy
ORDER BY
    csd.siteID, csd.yr, csd.doy

In [ ]:
flux[0:9]

The table `flux` contains one record for each day water chemistry was sampled, and the mean discharge rate from that day. Let's calculate the flux of DOC for each of those days:

In [ ]:
flux["flux"] = flux.dischargeMean*flux.doc
flux.flux = flux.flux.astype("float")

Now we have an estimate of the flux of DOC for a couple of dozen days per site per year. There are many directions an analysis could go at this point, let's pick something that can make a simple visualization. We'll find the maximum daily flux of DOC from each site across the dataset:

In [ ]:
flux_max = []
for i in list(dict.fromkeys(flux.siteID)):
    flux_max.append([i, max(flux.flux[flux["siteID"] == i])])
fluxmax = pandas.DataFrame(flux_max)
fluxmax.columns = ["siteID","docFlux"]

Using site coordinates retrieved from NEON's public API, and the `Plotly Express` module, we can map maximum daily DOC flux at NEON stream sites across the country:

In [ ]:
utmlist = []
for i in fluxmax.siteID:
    site_request = requests.get("https://data.neonscience.org/api/v0/locations/" + i)
    site_list = site_request.json()
    utmlist.append([i, site_list["data"]["locationDecimalLatitude"], 
                    site_list["data"]["locationDecimalLongitude"]])
utms = pandas.DataFrame(utmlist)
utms.columns = ["siteID","lat","lon"]

In [ ]:
utms["docFlux"] = fluxmax.docFlux/100
docUtm = utms[pandas.notna(utms["docFlux"])]

In [ ]:
fig = px.scatter_geo(docUtm, lat="lat", lon="lon", 
                     size="docFlux", scope="usa",
                     projection="albers usa",
                     color="siteID", size_max=150,
                     opacity=0.6)
fig.show()

The two large rivers in the southeast dominate the map; let's take a look without them:

In [ ]:
streamDoc = docUtm.drop(docUtm[(docUtm.siteID == 'BLWA') | (docUtm.siteID == 'FLNT')].index)

In [ ]:
fig = px.scatter_geo(streamDoc, lat="lat", lon="lon", 
                     size="docFlux", scope="usa",
                     projection="albers usa",
                     color="siteID", size_max=50,
                     opacity=0.6)
fig.show()

We can see the distribution of maximum DOC flux across the country.

### Final thoughts

A few notes on this demo. This was a simple exercise for illustration purposes, not a rigorous analysis. We calculated the DOC flux on each day when DOC was sampled, but the continuous discharge data are available on a 1-minute resolution. More typical for this type of analysis would be to build a model estimating the flux as a function of discharge, and use the high-resolution discharge data to extrapolate a total flux estimate for the entire year.

Similarly, there are many possible pathways to take regarding how much aggregation and data cleaning to carry out in the query step, as opposed to querying broadly and doing the data wrangling in the analysis phase. Choosing what to do in which part of the process depends on the nature of the analysis, and on familiarity with the data and data structures. In general, applying extensive data cleaning in the query step makes processing and analysis faster, by taking advantage of BigQuery's high speed performance and capabilities, but at the risk of missing critical nuance in the data. For example, if we didn't know to retain only the records where `dischargeFinalQF=0`, the daily mean discharge estimate from the query would include data that had been flagged as erroneous. As with all analyses, explore the data carefully before choosing an analytical approach.